In [ ]:
! pip install stable-baselines3[extra]
! pip install -q swig
! pip install -q gymnasium[box2d]
! apt-get update && apt-get install swig cmake

  Using cached ale_py-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.5
    Uninstalling ale-py-0.7.5:
      Successfully uninstalled ale-py-0.7.5
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 35.1 MB/s eta 0:00:00
y
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (190 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig 

In [ ]:
import shutil
import warnings
import gymnasium as gym
import cv2
import random
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

In [ ]:
!wget -O data.zip https://drive.google.com/uc?id=1JgkM0ot7kVKRXllM4cVLhb2IzEWbNFfb
shutil.unpack_archive("data.zip", "")

--2024-01-31 08:21:48--  https://drive.google.com/uc?id=1JgkM0ot7kVKRXllM4cVLhb2IzEWbNFfb
Resolving drive.google.com (drive.google.com)... 74.125.197.138, 74.125.197.100, 74.125.197.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.197.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1JgkM0ot7kVKRXllM4cVLhb2IzEWbNFfb [following]
--2024-01-31 08:21:48--  https://drive.usercontent.google.com/download?id=1JgkM0ot7kVKRXllM4cVLhb2IzEWbNFfb
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.20.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23362890 (22M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  22.28M  --.-KB/s    in 0.1s    

2024-01-31 08:21:52 (164 MB/s) - ‘data.

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("Test.mp4", fourcc, 30.0, (width, height))

episodes = 100
for episode in range(1, episodes+1):
    observation = env.reset()
    done = False
    score = 0

    while not done:
        action = random.choice([0, 1])
        observation, reward, done, info, _ = env.step(action)
        score += reward

        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        video_writer.write(frame)

    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()

Episode 1 score: 29.0
Episode 2 score: 18.0
Episode 3 score: 23.0
Episode 4 score: 11.0
Episode 5 score: 30.0
Episode 6 score: 30.0
Episode 7 score: 108.0
Episode 8 score: 19.0
Episode 9 score: 18.0
Episode 10 score: 34.0
Episode 11 score: 14.0
Episode 12 score: 22.0
Episode 13 score: 13.0
Episode 14 score: 12.0
Episode 15 score: 37.0
Episode 16 score: 13.0
Episode 17 score: 15.0
Episode 18 score: 15.0
Episode 19 score: 11.0
Episode 20 score: 19.0
Episode 21 score: 13.0
Episode 22 score: 17.0
Episode 23 score: 12.0
Episode 24 score: 47.0
Episode 25 score: 16.0
Episode 26 score: 45.0
Episode 27 score: 51.0
Episode 28 score: 17.0
Episode 29 score: 25.0
Episode 30 score: 39.0
Episode 31 score: 12.0
Episode 32 score: 45.0
Episode 33 score: 13.0
Episode 34 score: 26.0
Episode 35 score: 21.0
Episode 36 score: 23.0
Episode 37 score: 24.0
Episode 38 score: 13.0
Episode 39 score: 14.0
Episode 40 score: 22.0
Episode 41 score: 25.0
Episode 42 score: 38.0
Episode 43 score: 14.0
Episode 44 score: 9

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

model = PPO('MlpPolicy', env, verbose=1, tensorboard_log='training/logs')
model.learn(total_timesteps=100_000, log_interval=5, progress_bar=True)

model.save('training/trained_models/PPO_CartPole')
del model
model = PPO.load('training/trained_models/PPO_CartPole', env=env)

evaluate_policy(model, env, n_eval_episodes=10)

env = gym.make('CartPole-v1', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("CartPole.mp4", fourcc, 30.0, (width, height))

episodes = 100
for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        video_writer.write(frame)

    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()


Using cpu device
Logging to training/logs/PPO_4


Output()

-----------------------------------------
| time/                   |             |
|    fps                  | 955         |
|    iterations           | 5           |
|    time_elapsed         | 10          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.006657504 |
|    clip_fraction        | 0.0478      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.623      |
|    explained_variance   | 0.356       |
|    learning_rate        | 0.0003      |
|    loss                 | 18.9        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0135     |
|    value_loss           | 58.4        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 921          |
|    iterations           | 10           |
|    time_elapsed         | 22           |
|    total_timesteps      | 2

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Episode 1 score: [500.]
Episode 2 score: [500.]
Episode 3 score: [500.]
Episode 4 score: [500.]
Episode 5 score: [500.]
Episode 6 score: [500.]
Episode 7 score: [500.]
Episode 8 score: [500.]
Episode 9 score: [500.]
Episode 10 score: [500.]


In [ ]:
env = gym.make('LunarLander-v2', render_mode='rgb_array')

# 3.2 Train the model
model = DQN("MlpPolicy", env,
            batch_size=128,
            buffer_size=50_000,
            exploration_final_eps=0.1,
            exploration_fraction=0.12,
            gamma=0.99,
            gradient_steps=-1,
            learning_rate=0.00063,
            learning_starts=0,
            policy_kwargs=dict(net_arch=[256, 256]),
            target_update_interval=250,
            train_freq=4)

model.learn(total_timesteps=200_000, log_interval=50, progress_bar=True)

model.save('training/trained_models/DQN_LunarLander')
del model
model = DQN.load('training/trained_models/DQN_LunarLander')

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("Luna Lander.mp4", fourcc, 30.0, (width, height))

episodes = 10
for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        video_writer.write(frame)

    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()


Output()

mean_reward=197.96 +/- 75.47509636458136
Episode 1 score: [105.48331]
Episode 2 score: [260.66895]
Episode 3 score: [282.34128]
Episode 4 score: [43.053787]
Episode 5 score: [239.25487]
Episode 6 score: [243.17332]
Episode 7 score: [224.88861]
Episode 8 score: [238.78899]
Episode 9 score: [225.70004]
Episode 10 score: [66.24044]


In [ ]:
env = gym.make('CarRacing-v2', render_mode='rgb_array', verbose=0)
env = DummyVecEnv([lambda: env])

model = PPO("CnnPolicy", 'CarRacing-v2',
            tensorboard_log='training/logs',
            batch_size=128,
            clip_range=0.2,
            ent_coef=0.0,
            gae_lambda=0.95,
            gamma=0.99,
            learning_rate=0.0003,
            max_grad_norm=0.5,
            n_epochs=10)

model.learn(total_timesteps=400_000, log_interval=5, progress_bar=True)

model.save('training/trained_models/PPO_400kCarRacing_whp')
del model
model = PPO.load('training/trained_models/PPO_400kCarRacing_whp')

env = gym.make('CarRacing-v2', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

env = gym.make('CarRacing-v2', render_mode='rgb_array')
env = DummyVecEnv([lambda: env])

width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("Car Racing.mp4", fourcc, 30.0, (width, height))

episodes = 10
for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        video_writer.write(frame)

    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()


 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401,408/400,000  [ 2:48:37 < 0:00:00 , 46 it/s ]

mean_reward=427.54 +/- 164.74825671722698
Episode 1 score: [445.1347]
Episode 2 score: [312.18134]
Episode 3 score: [443.40515]
Episode 4 score: [197.94052]
Episode 5 score: [130.25812]
Episode 6 score: [793.11176]
Episode 7 score: [282.97452]
Episode 8 score: [345.7789]
Episode 9 score: [399.9969]
Episode 10 score: [496.37283]
